<a href="https://colab.research.google.com/github/haochu-liu/SML_practical/blob/main/knn_new" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
import random
random.seed(123)
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.preprocessing import StandardScaler

Read:
https://www.techscience.com/iasc/v34n3/47913/html


In [ ]:
# Load the training data and the test inputs
X = pd.read_csv('X_train.csv', index_col = 0, header=[0, 1, 2]) # inputs of the training set
y = pd.read_csv('y_train.csv', index_col = 0, squeeze=True).to_numpy() # outputs of the training set
X_test = pd.read_csv('X_test.csv', index_col = 0, header=[0, 1, 2]) # inputs of the test set
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

<ipython-input-4-4c2798697016>:3: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  y = pd.read_csv('y_train.csv', index_col = 0, squeeze=True).to_numpy() # outputs of the training set


In [ ]:
for i in np.unique(y_train):
    print(i, list(y_train).count(i))
# so we do not need to consider class imbalanaces

Electronic 589
Experimental 587
Folk 629
Hip-Hop 601
Instrumental 601
International 598
Pop 594
Rock 601


In [ ]:
lda = LinearDiscriminantAnalysis(shrinkage='auto', solver='eigen')
lda.fit(X_train, y_train)
lda.transform(X_train)

array([[ 0.17332673, -6.74845554,  1.20035769, ..., -4.99298111,
        -1.3793308 ,  0.29082656],
       [-1.78567969, -4.73824021, -0.9409046 , ..., -7.16208633,
         0.12207374,  2.51849543],
       [-4.23431512, -6.5695319 ,  2.83995857, ..., -7.3711081 ,
        -1.73886255,  1.38683477],
       ...,
       [-3.48529409, -7.62398749,  3.0242171 , ..., -7.39962503,
        -3.01799707,  0.45390228],
       [-4.24223563, -5.37546238,  0.53585725, ..., -5.47327871,
        -1.00172862,  0.61851824],
       [-2.69062026, -7.84755619,  1.90343903, ..., -5.91578503,
        -1.13550034, -1.70996844]])

In [ ]:
accuracies = np.zeros((2, 4))

for n_comps in range(4, 8):

    # Initialize variables to store results
    best_k = None
    best_accuracy = 0

    for k in [2, 5, 10, 25, 50, 75, 100, 200]:

        pipeline = Pipeline([('lda', LinearDiscriminantAnalysis(n_components=n_comps)),
                             ('knn', KNeighborsClassifier(n_neighbors=k))])

        # Use cross-validation to evaluate the accuracy
        accuracy_scores = cross_val_score(pipeline, X_train, y_train, cv=10, scoring='accuracy')

        # Calculate the average accuracy
        avg_accuracy = np.mean(accuracy_scores)

        # Update the best k if the current one is better
        if avg_accuracy > best_accuracy:
            best_k = k
            best_accuracy = avg_accuracy

        lda = LinearDiscriminantAnalysis(n_components=n_comps)
        lda.fit(X_train, y_train)
        X_train_lda = lda.transform(X_train)
        X_val_lda = lda.transform(X_val)

        best_knn_model = KNeighborsClassifier(n_neighbors=best_k)
        best_knn_model.fit(X_train_lda, y_train)
        accuracy_val = best_knn_model.score(X_val_lda, y_val)
        accuracies[0, n_comps-4] = best_k
        accuracies[1, n_comps-4] = accuracy_val

In [ ]:
accuracies

array([[50.        , 50.        , 75.        , 50.        ],
       [ 0.495     ,  0.53583333,  0.55      ,  0.55833333]])

In [ ]:
accuracies_ss = np.zeros((2, 4))

for n_comps in range(4, 8):

    # Initialize variables to store results
    best_k = None
    best_accuracy = 0

    for k in [2, 5, 10, 25, 50, 75, 100, 200]:

        pipeline = Pipeline([('lda', LinearDiscriminantAnalysis(n_components=n_comps)),
                             ('ss', StandardScaler()),
                             ('knn', KNeighborsClassifier(n_neighbors=k))])

        # Use cross-validation to evaluate the accuracy
        accuracy_scores = cross_val_score(pipeline, X_train, y_train, cv=10, scoring='accuracy')

        # Calculate the average accuracy
        avg_accuracy = np.mean(accuracy_scores)

        # Update the best k if the current one is better
        if avg_accuracy > best_accuracy:
            best_k = k
            best_accuracy = avg_accuracy

        lda = LinearDiscriminantAnalysis(n_components=n_comps)
        lda.fit(X_train, y_train)
        X_train_lda = lda.transform(X_train)
        X_val_lda = lda.transform(X_val)

        best_knn_model = KNeighborsClassifier(n_neighbors=best_k)
        best_knn_model.fit(X_train_lda, y_train)
        accuracy_val = best_knn_model.score(X_val_lda, y_val)
        accuracies_ss[0, n_comps-4] = best_k
        accuracies_ss[1, n_comps-4] = accuracy_val

In [ ]:
accuracies_ss

array([[ 75.        , 200.        ,  50.        ,  75.        ],
       [  0.49833333,   0.53916667,   0.56166667,   0.56166667]])

In [ ]:
accuracies_shrinkage = np.zeros((2, 4))

for n_comps in range(4, 8):

    # Initialize variables to store results
    best_k = None
    best_accuracy = 0

    for k in [2, 5, 10, 25, 50, 75, 100, 200]:

        pipeline = Pipeline([('lda', LinearDiscriminantAnalysis(n_components=n_comps,
                                                                solver='eigen', shrinkage='auto')),
                             ('knn', KNeighborsClassifier(n_neighbors=k))])

        # Use cross-validation to evaluate the accuracy
        accuracy_scores = cross_val_score(pipeline, X_train, y_train, cv=10, scoring='accuracy')

        # Calculate the average accuracy
        avg_accuracy = np.mean(accuracy_scores)

        # Update the best k if the current one is better
        if avg_accuracy > best_accuracy:
            best_k = k
            best_accuracy = avg_accuracy

        lda = LinearDiscriminantAnalysis(n_components=n_comps, solver='eigen', shrinkage='auto')
        lda.fit(X_train, y_train)
        X_train_lda = lda.transform(X_train)
        X_val_lda = lda.transform(X_val)

        best_knn_model = KNeighborsClassifier(n_neighbors=best_k)
        best_knn_model.fit(X_train_lda, y_train)
        accuracy_val = best_knn_model.score(X_val_lda, y_val)
        accuracies_shrinkage[0, n_comps-4] = best_k
        accuracies_shrinkage[1, n_comps-4] = accuracy_val

In [ ]:
accuracies_shrinkage

array([[ 75.        ,  50.        , 100.        ,  25.        ],
       [  0.51583333,   0.52583333,   0.545     ,   0.56083333]])

In [ ]:
accuracies_shrinkage_ss = np.zeros((2, 4))

for n_comps in range(4, 8):

    # Initialize variables to store results
    best_k = None
    best_accuracy = 0

    for k in [2, 5, 10, 25, 50, 75, 100, 200]:

        pipeline = Pipeline([('lda', LinearDiscriminantAnalysis(n_components=n_comps,
                                                                solver='eigen', shrinkage='auto')),
                             ('ss', StandardScaler()),
                             ('knn', KNeighborsClassifier(n_neighbors=k))])

        # Use cross-validation to evaluate the accuracy
        accuracy_scores = cross_val_score(pipeline, X_train, y_train, cv=10, scoring='accuracy')

        # Calculate the average accuracy
        avg_accuracy = np.mean(accuracy_scores)

        # Update the best k if the current one is better
        if avg_accuracy > best_accuracy:
            best_k = k
            best_accuracy = avg_accuracy

        lda = LinearDiscriminantAnalysis(n_components=n_comps, solver='eigen', shrinkage='auto')
        lda.fit(X_train, y_train)
        X_train_lda = lda.transform(X_train)
        X_val_lda = lda.transform(X_val)

        best_knn_model = KNeighborsClassifier(n_neighbors=best_k)
        best_knn_model.fit(X_train_lda, y_train)
        accuracy_val = best_knn_model.score(X_val_lda, y_val)
        accuracies_shrinkage_ss[0, n_comps-4] = best_k
        accuracies_shrinkage_ss[1, n_comps-4] = accuracy_val

In [ ]:
accuracies_shrinkage_ss

array([[100.        ,  25.        ,  50.        ,  50.        ],
       [  0.51666667,   0.5225    ,   0.54833333,   0.56666667]])

## Random forests

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
rfc.score(X_val, y_val)
rfc.feature_importances_

array([1.37165705e-03, 1.48410138e-03, 1.46189047e-03, 1.50502697e-03,
       1.38550051e-03, 1.81272278e-03, 1.34177172e-03, 1.58901102e-03,
       1.36988232e-03, 1.49131188e-03, 1.37890495e-03, 1.43901323e-03,
       1.62809879e-03, 1.62065054e-03, 1.46821659e-03, 1.39334862e-03,
       1.69158984e-03, 1.68137866e-03, 1.85347509e-03, 1.36731015e-03,
       1.45761181e-03, 2.01067020e-03, 1.35834787e-03, 1.75424013e-03,
       1.26388664e-03, 1.34778615e-03, 1.39347491e-03, 1.31917355e-03,
       1.12518519e-03, 1.12396067e-03, 1.33590560e-03, 1.05959575e-03,
       1.30536667e-03, 1.36193170e-03, 1.50200205e-03, 1.19518401e-03,
       1.28413696e-03, 1.24849413e-03, 1.54998732e-03, 1.31637967e-03,
       1.36470953e-03, 1.33395987e-03, 1.37332878e-03, 1.22432872e-03,
       1.37615275e-03, 1.41854487e-03, 1.38922863e-03, 1.29565118e-03,
       3.19797753e-04, 3.76864232e-04, 3.00112817e-04, 2.37338633e-04,
       3.03957714e-04, 2.96584587e-04, 3.78301642e-04, 4.25635567e-04,
      

In [ ]:
parameters = {'criterion':('gini', 'entropy'), 'max_depth':[3, 5, 10, 15, 20, 30],
              'n_estimators':[10, 20, 50, 100]}
rfc = RandomForestClassifier()
clf = GridSearchCV(rfc, parameters, scoring='accuracy')
clf.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
clf.cv_results_

{'mean_fit_time': array([ 0.37055106,  0.49494667,  1.20411673,  2.55913854,  0.42761245,
         0.86511593,  1.97086382,  3.98510571,  0.65059638,  1.64659743,
         3.33570046,  6.84950275,  0.75409489,  1.67851591,  3.90241494,
         7.98461919,  0.91048298,  1.55698519,  4.33665285,  8.13017793,
         0.952279  ,  1.54831161,  4.1608501 ,  8.08017507,  0.55194659,
         1.65818686,  3.18819466,  7.17831826,  0.86939788,  1.91473432,
         4.74032159,  9.59793534,  1.66701674,  3.05503016,  7.94847608,
        20.19301567,  3.05977216,  5.89507766,  8.40358939, 16.60135746,
         1.63186173,  3.32061038,  8.34485083, 16.54353929,  1.73010945,
         3.25574822,  8.6101944 , 16.33764081]),
 'std_fit_time': array([0.00799471, 0.01225048, 0.00920781, 0.2118279 , 0.05903378,
        0.13843243, 0.24886726, 0.27190159, 0.01158677, 0.28911795,
        0.34299999, 0.41276037, 0.00828237, 0.2276394 , 0.31975736,
        0.37004542, 0.16265085, 0.03307302, 0.4119095 , 0

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier()
clf.fit(X_train, y_train)
clf.score(X_val, y_val)

parameters = {'criterion':('gini', 'entropy'), 'max_depth':[3, 5, 10, 15, 20, 30],
              'n_estimators':[10, 20, 50, 100]}
rfc = RandomForestClassifier()
clf = GridSearchCV(rfc, parameters)
clf.fit(X_train, y_train)

AdaBoostClassifier()

In [ ]:
clf.score(X_val, y_val)

0.4141666666666667

#### Outlier detection?

In [ ]:
from sklearn.svm import OneClassSVM
outlierclf = OneClassSVM(kernel='poly')
outlierclf.fit(X_train, y_train)
a = outlierclf.predict(X_train)
collections.Counter(a)
a = outlierclf.predict(X_train)
collections.Counter(a)

OneClassSVM(kernel='poly')

In [ ]:
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
kernel = 1.0 * RBF(1.0)
gpc = GaussianProcessClassifier(kernel=kernel, random_state=0).fit(X_train, y_train)